In [12]:
import openai
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [13]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [17]:
model = ChatOpenAI(api_key=openai.api_key, model="gpt-3.5-turbo", temperature=0.7)

In [18]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")

In [19]:
chain = prompt | model

In [22]:
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")